In [1]:
import numpy as np

from preprocessing import obtain_chembl_data,combine_chembl_papyrus_mutants,merge_chembl_papyrus_mutants
from annotation import *

### Annotation workflow step by step

In [2]:
# 1. Get ChEMBL data using ChEMBl downloader
chembl_data = obtain_chembl_data(chembl_version='31', chunksize= 100_000)
chembl_data

C:\Users\gorostiolam\AppData\Local\Continuum\anaconda3\envs\mutants-in-pcm\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (4,6,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  


,description,assay_id,variant_id,assay_chembl_id,assay_organism,year,abstract,mutation,activity_id,pchembl_value,standard_type,activity_comment,chembl_id,canonical_smiles,accession,sequence,organism
0,In vivo inhibitory activity against human Hepa...,83907,NaN,CHEMBL872937,NaN,2004.0,"A novel class of 2,3-dihydro-1,3-dioxo-1H-isoi...",NaN,31864,5.60,IC50,NaN,CHEMBL324340,Cc1ccc2oc(-c3cccc(N4C(=O)c5ccc(C(=O)O)cc5C4=O)...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,Homo sapiens
1,In vivo inhibitory activity against human Hepa...,83907,NaN,CHEMBL872937,NaN,2004.0,"A novel class of 2,3-dihydro-1,3-dioxo-1H-isoi...",NaN,31866,5.05,IC50,NaN,CHEMBL109600,COc1ccccc1-c1ccc2oc(-c3ccc(OC)c(N4C(=O)c5ccc(C...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,Homo sapiens
2,Inhibitory activity against Palmitoyl-CoA oxid...,154606,NaN,CHEMBL760688,NaN,2004.0,New inhibitors of palmitoylCoA oxidation were ...,NaN,31868,5.40,IC50,NaN,CHEMBL357278,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4ccc(Cl)c(C(...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,Rattus norvegicus
3,Inhibitory activity against Palmitoyl-CoA oxid...,154606,NaN,CHEMBL760688,NaN,2004.0,New inhibitors of palmitoylCoA oxidation were ...,NaN,31870,4.77,IC50,NaN,CHEMBL357119,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)NCCc4ccccc4)CC...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,Rattus norvegicus
4,Inhibitory activity against Palmitoyl-CoA oxid...,154606,NaN,CHEMBL760688,NaN,2004.0,New inhibitors of palmitoylCoA oxidation were ...,NaN,31872,6.75,IC50,NaN,CHEMBL152968,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccc...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,Rattus norvegicus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270179,KinomeScan assay: inhibition of LATS2,2136225,NaN,CHEMBL4879778,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347581,7.26,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,Homo sapiens
5270180,KinomeScan assay: inhibition of ROCK1,2136385,NaN,CHEMBL4879938,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347583,7.01,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,Homo sapiens
5270181,KinomeScan assay: inhibition of ROCK2,2136386,NaN,CHEMBL4879939,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347585,7.09,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,Homo sapiens
5270182,KinomeScan assay: inhibition of TGFBR2,2136430,NaN,CHEMBL4879983,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347587,7.27,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,Homo sapiens


In [3]:
# 2. Get unique assays from ChEMBL
chembl_assays = filter_assay_data(chembl_data)
chembl_assays

,assay_id,description,accession,sequence,mutation
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN
2,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN
5,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN
6,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN
7,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN
...,...,...,...,...,...
5270179,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN
5270180,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN
5270181,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN
5270182,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN


In [4]:
# 3. Extract mutation-looking ReGex patterns from assay descriptions and store them in column aa_change
# Note: column mutation contains annotated ChEMBL variants
chembl_assays_extracted = extract_aa_change(chembl_assays)
chembl_assays_extracted

,assay_id,description,accession,sequence,mutation,aa_change
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[]
2,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[]
5,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[]
6,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[]
7,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[]
...,...,...,...,...,...,...
5270179,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[]
5270180,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[]
5270181,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[]
5270182,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[]


In [5]:
# Check how many mutations were extracted with ReGex expressions
chembl_assays_extracted[chembl_assays_extracted.apply(lambda x: len(x['aa_change']) > 0, axis=1)]

,assay_id,description,accession,sequence,mutation,aa_change
178,59914,Binding affinity to cloned human Dopamine rece...,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,NaN,[A9L]
293,106820,Agonist activity at human melanocortin recepto...,P32245,MVNSTHRGMHTSLHLWNRSSYRLHSNASESLGKGYSDGGCYEQLFV...,NaN,[C4R]
294,106816,Agonistic activity against human melanocortin ...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R]
296,106815,Agonist activity at human melanocortin recepto...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R]
499,2560,Ability to stimulate phosphoinositide hydrolys...,P14842,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,NaN,[H3T]
...,...,...,...,...,...,...
5270146,2135964,Inhibition of human PI3K p110beta/p85alpha by ...,P42338,MCFSFIMPPAMADILDIWAVDSQIASDGSIPVDFLLPTGIYIQLEV...,NaN,[I3K]
5270149,2135965,Inhibition of human PI3K p110delta/p85alpha by...,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,NaN,[I3K]
5270150,2135965,Inhibition of human PI3K p110delta/p85alpha by...,O00329,MPPGVDCPMEFWTKEENQSVVVDFLLPTGVYLNFPVSRNANLSTIK...,NaN,[I3K]
5270153,2135966,Inhibition of human PI3Kgamma by HTRF assay,P48736,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,NaN,[I3K]


In [6]:
# 4. Visualize exceptions defined by matches with cell lines, targets and genes
# This step is included in validate_aa_change()
exceptions = define_aa_change_exceptions(chembl_assays_extracted, chembl_version='31')
print_exceptions =  exceptions[exceptions.apply(lambda x: len(x['exception_reasons']) > 0, axis=1)]
print_exceptions

,assay_id,description,accession,sequence,mutation,aa_change,exception_flags,exception_reasons
84,59914,Binding affinity to cloned human Dopamine rece...,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,NaN,[A9L],[True],[assay_cell_type]
174,106820,Agonist activity at human melanocortin recepto...,P32245,MVNSTHRGMHTSLHLWNRSSYRLHSNASESLGKGYSDGGCYEQLFV...,NaN,[C4R],[True],[component_synonym]
175,106816,Agonistic activity against human melanocortin ...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
176,106815,Agonist activity at human melanocortin recepto...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
312,2560,Ability to stimulate phosphoinositide hydrolys...,P14842,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,NaN,[H3T],[True],[cell_name]
...,...,...,...,...,...,...,...,...
376205,2135964,Inhibition of human PI3K p110beta/p85alpha by ...,P42338,MCFSFIMPPAMADILDIWAVDSQIASDGSIPVDFLLPTGIYIQLEV...,NaN,[I3K],[True],[pref_name]
376206,2135965,Inhibition of human PI3K p110delta/p85alpha by...,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,NaN,[I3K],[True],[component_synonym]
376207,2135965,Inhibition of human PI3K p110delta/p85alpha by...,O00329,MPPGVDCPMEFWTKEENQSVVVDFLLPTGVYLNFPVSRNANLSTIK...,NaN,[I3K],[True],[component_synonym]
376208,2135966,Inhibition of human PI3Kgamma by HTRF assay,P48736,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,NaN,[I3K],[True],[component_synonym]


In [7]:
# Check how many extracted mutations had exceptions defined
exceptions[exceptions.apply(lambda x: True in x['exception_flags'], axis=1) & exceptions.apply(lambda x: len(x['aa_change']) > 0, axis=1)]

,assay_id,description,accession,sequence,mutation,aa_change,exception_flags,exception_reasons
84,59914,Binding affinity to cloned human Dopamine rece...,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,NaN,[A9L],[True],[assay_cell_type]
174,106820,Agonist activity at human melanocortin recepto...,P32245,MVNSTHRGMHTSLHLWNRSSYRLHSNASESLGKGYSDGGCYEQLFV...,NaN,[C4R],[True],[component_synonym]
175,106816,Agonistic activity against human melanocortin ...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
176,106815,Agonist activity at human melanocortin recepto...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
312,2560,Ability to stimulate phosphoinositide hydrolys...,P14842,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,NaN,[H3T],[True],[cell_name]
...,...,...,...,...,...,...,...,...
376205,2135964,Inhibition of human PI3K p110beta/p85alpha by ...,P42338,MCFSFIMPPAMADILDIWAVDSQIASDGSIPVDFLLPTGIYIQLEV...,NaN,[I3K],[True],[pref_name]
376206,2135965,Inhibition of human PI3K p110delta/p85alpha by...,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,NaN,[I3K],[True],[component_synonym]
376207,2135965,Inhibition of human PI3K p110delta/p85alpha by...,O00329,MPPGVDCPMEFWTKEENQSVVVDFLLPTGVYLNFPVSRNANLSTIK...,NaN,[I3K],[True],[component_synonym]
376208,2135966,Inhibition of human PI3Kgamma by HTRF assay,P48736,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,NaN,[I3K],[True],[component_synonym]


In [8]:
# 5. Validate extracted mutations based on the previously defined exceptions and their WT sequence
chembl_assays_validated = validate_aa_change(chembl_assays_extracted, chembl_version='31')
chembl_assays_validated

,assay_id,description,accession,sequence,mutation,aa_change,mutants
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[],[]
1,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[],[]
2,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[],[]
3,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[],[]
4,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[],[]
...,...,...,...,...,...,...,...
376228,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[],[]
376229,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[],[]
376230,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[],[]
376231,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[],[]


In [9]:
# 6. Based on the validated mutants, create Papyrus-like identifiers
chembl_assays_annotated = create_papyrus_columns(chembl_assays_validated)
chembl_assays_annotated

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[],[],Q9Y251_WT,WT
1,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[],[],P07872_WT,WT
2,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[],[],P05177_WT,WT
3,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[],[],P08684_WT,WT
4,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[],[],P11712_WT,WT
...,...,...,...,...,...,...,...,...,...
376228,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[],[],Q9NRM7_WT,WT
376229,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[],[],Q13464_WT,WT
376230,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[],[],O75116_WT,WT
376231,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[],[],P37173_WT,WT


In [10]:
# 7. Map newly annotated assays to the ChEMBL bioactivity dataset
chembl_bioactivity_dataset = map_activity_mutations(chembl_data, chembl_assays_annotated)
chembl_bioactivity_dataset

,target_id,chembl_id,assay_id,activity_id,accession,pchembl_value,activity_comment,canonical_smiles,mutation,year,sequence,pchembl_value_Mean,Activity_class
0,A0A024AXB9_WT,CHEMBL1230673,[1986298],[20667385],A0A024AXB9,[6.37],[nan],CN[C@@H](C)C(=O)N[C@H](C(=O)N1CC[C@H](C)[C@H]1...,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,6.370000,NaN
1,A0A024AXB9_WT,CHEMBL1332616,"[1986298, 1986301]","[20667410, 20667411]",A0A024AXB9,"[5.77, 6.22]","[nan, nan]",O=C1OC2(c3ccccc31)c1cc(I)c(O)c(I)c1Oc1c2cc(I)c...,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,5.995000,NaN
2,A0A024AXB9_WT,CHEMBL151,[1855280],[19053938],A0A024AXB9,[4.28],[nan],O=c1cc(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,4.280000,NaN
3,A0A024AXB9_WT,CHEMBL1535,[1986298],[20667415],A0A024AXB9,[4.04],[nan],CCN(CCO)CCCC(C)Nc1ccnc2cc(Cl)ccc12,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,4.040000,NaN
4,A0A024AXB9_WT,CHEMBL164,"[1855280, 1855317, 1855317]","[19053935, 19053936, 19053937]",A0A024AXB9,"[4.66, 5.89, 6.1]","[nan, nan, nan]",O=c1c(O)c(-c2cc(O)c(O)c(O)c2)oc2cc(O)cc(O)c12,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,5.550000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565038,nan_WT,CHEMBL529,"[582849, 582849, 582849]","[2702686, 2702686, 2702686]",NaN,"[6.52, 6.52, 6.52]","[nan, nan, nan]",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2009.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,6.520000,NaN
2565039,nan_WT,CHEMBL532,"[576183, 576183, 576183, 576182, 576182, 57618...","[2663798, 2663798, 2663798, 2663799, 2663799, ...",NaN,"[7.54, 7.54, 7.54, 7.96, 7.96, 7.96, 5.72, 5.7...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2007.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,6.418889,NaN
2565040,nan_WT,CHEMBL553222,"[582849, 582849, 582849]","[2702683, 2702683, 2702683]",NaN,"[7.0, 7.0, 7.0]","[nan, nan, nan]",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2009.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,7.000000,NaN
2565041,nan_WT,CHEMBL557931,"[582849, 582849, 582849]","[2702682, 2702682, 2702682]",NaN,"[5.89, 5.89, 5.89]","[nan, nan, nan]",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2009.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,5.890000,NaN


In [11]:
# Check how many bioactivity datapoints in ChEMBL are tested in mutants
chembl_bioactivity_dataset[~chembl_bioactivity_dataset['target_id'].str.contains('WT')]

,target_id,chembl_id,assay_id,activity_id,accession,pchembl_value,activity_comment,canonical_smiles,mutation,year,sequence,pchembl_value_Mean,Activity_class
257,A0A045ISB3_F161S,CHEMBL4637373,[1992590],[20694452],A0A045ISB3,[5.13],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.130,NaN
258,A0A045ISB3_F161S,CHEMBL4637459,[1992590],[20694450],A0A045ISB3,[5.07],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.070,NaN
259,A0A045ISB3_F161S,CHEMBL4638011,[1992590],[20694453],A0A045ISB3,[5.04],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.040,NaN
260,A0A045ISB3_F161S,CHEMBL4647367,[1992590],[20694451],A0A045ISB3,[5.27],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.270,NaN
261,A0A045ISB3_I203A,CHEMBL4637373,[1992587],[20694440],A0A045ISB3,[7.68],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,I203A,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,7.680,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562407,Q9YQ12_V82F,CHEMBL3797292,"[1576372, 1576372, 1822317, 1822317]","[16572620, 16572621, 18916325, 18916333]",Q9YQ12,"[7.62, 7.89, 7.62, 7.89]","[nan, nan, nan, nan]",COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,V82F,2016.0,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.755,NaN
2562408,Q9YQ12_V82F,CHEMBL3799941,"[1576372, 1576372, 1822317, 1822317]","[16572618, 16572619, 18916332, 18916379]",Q9YQ12,"[7.72, 8.0, 8.0, 7.72]","[nan, nan, nan, nan]",COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,V82F,2016.0,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.860,NaN
2562409,Q9YQ12_V82F,CHEMBL4452050,[1854379],[19051909],Q9YQ12,[8.77],[nan],COc1ccc(S(=O)(=O)N(CC(C)C)C[C@H](O)[C@@H](Cc2c...,V82F,2019.0,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,8.770,NaN
2564617,T1QYY3_G140S_Q148H,CHEMBL4779601,[2070418],[22876490],T1QYY3,[7.77],[nan],CN(C)C(=O)C(=O)NC12CCC(CC1)Cn1c2nc(C(=O)NCc2cc...,"G140S,Q148H",2020.0,FLDGIDKAQEDHEKYHSNWRAMASDFNMPPIXAKEIVASCDKCQQK...,7.770,NaN


In [12]:
# 8. Concatenate newly annotated ChEMBL bioactivity data with non-ChEMBL part of Papyrus (only targets with at least
# one annotated variant)
chembl_papyrus_bioactivity_dataset = combine_chembl_papyrus_mutants('31', '05.5', 'nostereo', 1_000_000)
chembl_papyrus_bioactivity_dataset

C:\Users\gorostiolam\AppData\Local\Continuum\anaconda3\envs\mutants-in-pcm\lib\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


,CID,connectivity,target_id,AID,accession,pchembl_value_Mean,SMILES,sequence,source,Activity_class,Year
0,CHEMBL4637373,LUVQLVJTOZVLDB,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.130,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
1,CHEMBL4637459,ALIJNJWQOZKBPP,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.070,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
2,CHEMBL4638011,IPTNWUSXZLVWFT,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.040,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
3,CHEMBL4647367,CXXIUJZSLPZZIM,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.270,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
4,CHEMBL4637373,LUVQLVJTOZVLDB,A0A045ISB3_I203A,[1992587],A0A045ISB3,7.680,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
...,...,...,...,...,...,...,...,...,...,...,...
646472,Christmann2016.compound.1413,VMJFTOSOFDEKTM,P08581_Y1230H,Christmann2016.assay,P08581,9.222,Cn1cc(-c2cnc3ccc4c(cc(CS(=O)(=O)NCc5ncccc5)cc4...,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,Papyrus05.5_Christmann2016,NaN,2011.0
646473,Christmann2016.compound.1640,WCIGMFCFPXZRMQ,Q5S007_A2016T,Christmann2016.assay,Q5S007,7.091,O=C(Nc1cnccc1)c1cc(-c2ccnc(F)c2)ccc1OCc1ccccc1,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Papyrus05.5_Christmann2016,NaN,2012.0
646474,Christmann2016.compound.1392,YEVOZZZLKJKCCD,Q5S007_A2016T,Christmann2016.assay,Q5S007,6.813,CNc1nc(Nc2c(OC)cc(C(=O)N3CCOCC3)cc2)ncc1Cl,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Papyrus05.5_Christmann2016,NaN,2012.0
646475,Christmann2016.compound.1640,WCIGMFCFPXZRMQ,Q5S007_G2019S_A2016T,Christmann2016.assay,Q5S007,7.213,O=C(Nc1cnccc1)c1cc(-c2ccnc(F)c2)ccc1OCc1ccccc1,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Papyrus05.5_Christmann2016,NaN,2012.0


In [13]:
# 9. Merge ChEMBL and Papyrus data by calculating mean pChEMBL values (only targets with at least one annotated variant)
combined_dataset = merge_chembl_papyrus_mutants('31', '05.5', 'nostereo', 1_000_000)
combined_dataset

,target_id,connectivity,pchembl_value_Mean,Activity_class_consensus,source,SMILES,CID,accession,sequence,Year
0,A0A045ISB3_F161S,ALIJNJWQOZKBPP,5.07,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
1,A0A045ISB3_F161S,CXXIUJZSLPZZIM,5.27,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,CHEMBL4647367,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
2,A0A045ISB3_F161S,IPTNWUSXZLVWFT,5.04,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,CHEMBL4638011,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
3,A0A045ISB3_F161S,LUVQLVJTOZVLDB,5.13,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,CHEMBL4637373,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
4,A0A045ISB3_I203A,ALIJNJWQOZKBPP,7.37,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
...,...,...,...,...,...,...,...,...,...,...
620050,Q9YQ12_WT,ZYLARFCKPNSSDA,6.17,"Series([], dtype: object)",ChEMBL31,CC(C)[C@H](NC(=O)COc1ccccc1)C(=O)N[C@@H](Cc1cc...,CHEMBL3331342,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2014.0
620051,Q9YQ12_WT,ZYLFWZWRSRMGBQ,10.10,"Series([], dtype: object)",ChEMBL31,Nc1cccc(CN2C(=O)N(Cc3ccc4cn[nH]c4c3)C(Cc3ccccc...,CHEMBL2296983,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2013.0
620052,Q9YQ12_WT,ZZCUJLFFPCGLAL,5.51,"Series([], dtype: object)",ChEMBL31,CC1(C)CC[C@]2(C(=O)NCCCCCC(=O)NCC(=O)O)CC[C@]3...,CHEMBL4530451,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2019.0
620053,Q9YQ12_WT,ZZGMOZUZSAJAML,6.26,"Series([], dtype: object)",ChEMBL31,CC(C)c1nc(CN(C(=O)N[C@H](C(=O)N[C@H](CC[C@H](C...,CHEMBL3115161,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2014.0


In [14]:
# Check how many bioactivity datapoints in ChEMBL + Papyrus are tested in mutants
combined_dataset[~combined_dataset['target_id'].str.contains('WT')]

,target_id,connectivity,pchembl_value_Mean,Activity_class_consensus,source,SMILES,CID,accession,sequence,Year
0,A0A045ISB3_F161S,ALIJNJWQOZKBPP,5.070,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
1,A0A045ISB3_F161S,CXXIUJZSLPZZIM,5.270,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,CHEMBL4647367,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
2,A0A045ISB3_F161S,IPTNWUSXZLVWFT,5.040,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,CHEMBL4638011,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
3,A0A045ISB3_F161S,LUVQLVJTOZVLDB,5.130,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,CHEMBL4637373,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
4,A0A045ISB3_I203A,ALIJNJWQOZKBPP,7.370,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0
...,...,...,...,...,...,...,...,...,...,...
618958,Q9YQ12_V82A,SBBYGVNSLWBXFB,7.455,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,CHEMBL3799941,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2016.0
618959,Q9YQ12_V82A,XHKGWTXTLQTTQA,7.400,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,CHEMBL3797292,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2016.0
618960,Q9YQ12_V82F,MEWAZRJLRMEJDV,8.770,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)C[C@H](O)[C@@H](Cc2c...,CHEMBL4452050,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2019.0
618961,Q9YQ12_V82F,SBBYGVNSLWBXFB,7.860,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,CHEMBL3799941,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2016.0


In [15]:
# Check the distribution of per source of mutant bioactivity datapoints
combined_dataset[~combined_dataset['target_id'].str.contains('WT')].groupby(['source'])['pchembl_value_Mean'].count()

source
ChEMBL31                               22594
Papyrus05.5_Christmann2016               706
Papyrus05.5_Christmann2016;ChEMBL31     1422
Name: pchembl_value_Mean, dtype: int64

In [16]:
# Check how many bioactivity datapoints in ChEMBL + Papyrus are tested in mutants (BEFORE MERGING)
chembl_papyrus_bioactivity_dataset[~chembl_papyrus_bioactivity_dataset['target_id'].str.contains('WT')]

,CID,connectivity,target_id,AID,accession,pchembl_value_Mean,SMILES,sequence,source,Activity_class,Year
0,CHEMBL4637373,LUVQLVJTOZVLDB,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.130,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
1,CHEMBL4637459,ALIJNJWQOZKBPP,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.070,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
2,CHEMBL4638011,IPTNWUSXZLVWFT,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.040,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
3,CHEMBL4647367,CXXIUJZSLPZZIM,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.270,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
4,CHEMBL4637373,LUVQLVJTOZVLDB,A0A045ISB3_I203A,[1992587],A0A045ISB3,7.680,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
...,...,...,...,...,...,...,...,...,...,...,...
646472,Christmann2016.compound.1413,VMJFTOSOFDEKTM,P08581_Y1230H,Christmann2016.assay,P08581,9.222,Cn1cc(-c2cnc3ccc4c(cc(CS(=O)(=O)NCc5ncccc5)cc4...,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,Papyrus05.5_Christmann2016,NaN,2011.0
646473,Christmann2016.compound.1640,WCIGMFCFPXZRMQ,Q5S007_A2016T,Christmann2016.assay,Q5S007,7.091,O=C(Nc1cnccc1)c1cc(-c2ccnc(F)c2)ccc1OCc1ccccc1,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Papyrus05.5_Christmann2016,NaN,2012.0
646474,Christmann2016.compound.1392,YEVOZZZLKJKCCD,Q5S007_A2016T,Christmann2016.assay,Q5S007,6.813,CNc1nc(Nc2c(OC)cc(C(=O)N3CCOCC3)cc2)ncc1Cl,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Papyrus05.5_Christmann2016,NaN,2012.0
646475,Christmann2016.compound.1640,WCIGMFCFPXZRMQ,Q5S007_G2019S_A2016T,Christmann2016.assay,Q5S007,7.213,O=C(Nc1cnccc1)c1cc(-c2ccnc(F)c2)ccc1OCc1ccccc1,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Papyrus05.5_Christmann2016,NaN,2012.0


In [17]:
# Check the distribution of per source of mutant bioactivity datapoints (BEFORE MERGING)
chembl_papyrus_bioactivity_dataset[~chembl_papyrus_bioactivity_dataset['target_id'].str.contains('WT')].groupby\
    (['source']).count()

,CID,connectivity,target_id,AID,accession,pchembl_value_Mean,SMILES,sequence,Activity_class,Year
source,,,,,,,,,,
ChEMBL31,24733,24733,24733,24733,24733,24733,24733,24733,0,24198
Papyrus05.5_Christmann2016,2128,2128,2128,2128,2128,2128,2128,2128,0,2128


#### Get an example assay for Figure 1 of the MS
Where two possible mutations were extracted, one of them being validated in the end and the other one being defined
as an exception

In [18]:
# Visualize assays with 2 exceptions defined and no previously ChEMBL annotated variants
exceptions[exceptions.apply(lambda x: len(x['exception_reasons']) == 2, axis=1) & exceptions["mutation"].isnull()]

,assay_id,description,accession,sequence,mutation,aa_change,exception_flags,exception_reasons
329,231739,Selectivity ratio of hA2A to hA2B,P29275,MLLETQDALYVALELVIAALSVAGNVLVCAAVGTANTLQTPTNYFL...,NaN,"[A2A, A2B]","[True, True]","[component_synonym, component_synonym]"
330,231739,Selectivity ratio of hA2A to hA2B,P29274,MPIMGSSVYITVELAIAVLAILGNVLVCWAVWLNSNLQNVTNYFVV...,NaN,"[A2A, A2B]","[True, True]","[component_synonym, component_synonym]"
394,147717,Evaluated for agonist activity against phospho...,P49652,MTEALISAALNGTQPELLAGGWAAGNASTKCSLTKTGFQFYYLPTV...,NaN,"[P2Y, P2Y]","[True, True]","[pref_name, pref_name]"
395,147736,Agonist activity against phospholipase C coupl...,Q96G91,MAANVSGAKSCPANFLAAADDKLSGFQGDFLWPILVVEFLVAVASN...,NaN,"[P2Y, P2Y]","[True, True]","[pref_name, pref_name]"
396,147742,Antagonist activity against phospholipase C co...,Q9EPX4,MEVPGANATSANTTSIPGTSTLCSRDYKITQVLFPLLYTVLFFAGL...,NaN,"[P2Y, P2Y]","[True, True]","[pref_name, pref_name]"
...,...,...,...,...,...,...,...,...
375993,2135261,Inhibition of N-terminal His6-tagged KDM4E (1 ...,B2RXH2,MKSVHSSPQNTSHTIMTFYPTMEEFADFNTYVAYMESQGAHQAGLA...,NaN,"[M4E, H3K]","[True, False]","[component_synonym, None]"
375994,2135262,Inhibition of N-terminal His6-tagged KDM4E (1 ...,B2RXH2,MKSVHSSPQNTSHTIMTFYPTMEEFADFNTYVAYMESQGAHQAGLA...,NaN,"[M4E, H3K]","[True, False]","[component_synonym, None]"
375995,2135263,Inhibition of N-terminal His6-tagged KDM4E (1 ...,B2RXH2,MKSVHSSPQNTSHTIMTFYPTMEEFADFNTYVAYMESQGAHQAGLA...,NaN,"[M4E, H3K]","[True, False]","[component_synonym, None]"
375996,2135264,Inhibition of N-terminal His6-tagged KDM4E (1 ...,B2RXH2,MKSVHSSPQNTSHTIMTFYPTMEEFADFNTYVAYMESQGAHQAGLA...,NaN,"[M4E, H3K]","[True, False]","[component_synonym, None]"


In [19]:
# Manually selected assay ID for Figure 1`based on options above
fig1_assay_id = 306767

In [20]:
# Assay description
print(str(exceptions[exceptions["assay_id"] == fig1_assay_id].description.tolist()[0]))

Inhibition concentration required against human Y306F mutant strain P2Y1 receptor expressed in COS-7 cells is determined using [3H]MRS2279 as radioligand


In [21]:
# Assay data
exceptions[exceptions["assay_id"] == fig1_assay_id]

,assay_id,description,accession,sequence,mutation,aa_change,exception_flags,exception_reasons
90500,306767,Inhibition concentration required against huma...,P47900,MTEVLWPAVPNGTDAAFLAGPGSSWGNSTVASTAAVSSSFKCALTK...,NaN,"[Y306F, P2Y]","[False, True]","[None, pref_name]"


In [22]:
# Bioactivity data available for the mutant in the defiend assay
chembl_bioactivity_dataset[chembl_bioactivity_dataset["target_id"] == "P47900_Y306F"]

,target_id,chembl_id,assay_id,activity_id,accession,pchembl_value,activity_comment,canonical_smiles,mutation,year,sequence,pchembl_value_Mean,Activity_class
1620642,P47900_Y306F,CHEMBL129841,[306767],[1474155],P47900,[6.01],[nan],CNc1ncnc2c1ncn2[C@H]1C[C@H](OP(=O)(O)O)[C@@H](...,NaN,2004.0,MTEVLWPAVPNGTDAAFLAGPGSSWGNSTVASTAAVSSSFKCALTK...,6.01,NaN
1620643,P47900_Y306F,CHEMBL435402,"[306767, 304377]","[1473884, 1473905]",P47900,"[6.44, 7.72]","[nan, nan]",CSc1nc(N)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(=O)(...,NaN,2004.0,MTEVLWPAVPNGTDAAFLAGPGSSWGNSTVASTAAVSSSFKCALTK...,7.08,NaN


#### Annotation statistics

In [23]:
# Check for how many assay-accession pairs we have extracted and validated mutants
chembl_mutants_extracted = chembl_assays_annotated[chembl_assays_annotated.apply(lambda x: x['Protein_Type'] != 'WT', axis=1)]
chembl_mutants_extracted

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
2008,164502,Inhibitory activity against HIV-1 reverse tran...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184V,[M184V],[M184V],Q72547_M184V,M184V
2009,164500,Inhibitory activity against HIV-1 reverse tran...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184I,[M184I],[M184I],Q72547_M184I,M184I
2163,164503,Substrate Activity against HIV-1 reverse trans...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184V,[M184V],[M184V],Q72547_M184V,M184V
2164,164501,Substrate Activity against HIV-1 reverse trans...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184I,[M184I],[M184I],Q72547_M184I,M184I
3102,205569,Binding affinity against Gln165Ala mutant type...,P25103,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...,Q165A,[Q165A],[Q165A],P25103_Q165A,Q165A
...,...,...,...,...,...,...,...,...,...
375936,2135067,Protac activity at CRBN/AR T878A mutant in hum...,P10275,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,T878A,[T878A],[T878A],P10275_T878A,T878A
375967,2135214,Inhibition of EGFR L858R mutant (unknown origin),P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,L858R,[L858R],[L858R],P00533_L858R,L858R
376193,2135936,Inhibition of GDP-loaded His-tagged KRAS G12C ...,P01116,MTEYKLVVVGACGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,G12C,[G12C],[G12C],P01116_G12C,G12C
376194,2135937,Inhibition of KRAS G12C mutant in human NCI-H3...,P01116,MTEYKLVVVGACGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,G12C,[G12C],[G12C],P01116_G12C,G12C


In [24]:
# Check how many of the extracted mutants were not previously defined in ChEMBL's variant variable 'mutation'
chembl_new_mutants_extracted1 = chembl_mutants_extracted[(chembl_mutants_extracted['mutation'].isna())]
chembl_new_mutants_extracted1

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
3103,226874,Ratio of IC50 value against hNK1 receptor to t...,P25103,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...,NaN,[Q165A],[Q165A],P25103_Q165A,Q165A
5557,153379,Transcriptional activation activity on human I...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[I272F],[I272F],Q07869_I272F,I272F
5558,153380,Transcriptional activation activity on human T...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[T279M],[T279M],Q07869_T279M,T279M
5559,153557,Ratio of transcriptional activation of I272F m...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[I272F],[I272F],Q07869_I272F,I272F
5560,153560,Ratio of transcriptional activation of T279M m...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[T279M],[T279M],Q07869_T279M,T279M
...,...,...,...,...,...,...,...,...,...
371084,2113388,Inhibition of human non-phosphorylated ABL1 F3...,P00519,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,NaN,[F317L],[F317L],P00519_F317L,F317L
373669,2122982,Inhibition of MLL1 (unknown origin)-mediated H...,Q03164,MAKSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,NaN,[H3K],[H3K],Q03164_H3K,H3K
374059,2125019,Activation of human PKM2-C424A expressed in Es...,P14618,MSKPHSEAGTAFIQTQQLHAAMADTFLEHMCRLDIDSPPITARNTG...,NaN,[C424A],[C424A],P14618_C424A,C424A
374624,2127601,Binding affinity to human partial length MCL1 ...,Q07820,MFGLKRNAVIGLNLYCGGAGLGAGSGGATRPGGRLLATEKEASARR...,NaN,"[D172N, L2S]",[D172N],Q07820_D172N,D172N


In [25]:
# Check how many of the extracted mutants were defined as mutated by the actual mutation was not specified on ChEMBL
chembl_new_mutants_extracted2 = chembl_mutants_extracted[chembl_mutants_extracted['mutation'] == 'UNDEFINED MUTATION']
chembl_new_mutants_extracted2

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
225249,1433633,Potentiation of human CFTR F508del/G551D mutan...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,UNDEFINED MUTATION,[G551D],[G551D],P13569_G551D,G551D
227270,1442036,Inhibition of KIT V559/T670I mutant (unknown o...,P10721,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,UNDEFINED MUTATION,[T670I],[T670I],P10721_T670I,T670I
228081,1435798,Inhibition of wild type HIV1 reverse transcrip...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,UNDEFINED MUTATION,"[Y181I, Y181C]","[Y181C, Y181I]",Q72547_Y181C_Y181I,Y181C;Y181I
228390,1440950,Inhibition of EGFR T790M/del746 to 750 mutant ...,P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,UNDEFINED MUTATION,[T790M],[T790M],P00533_T790M,T790M
228616,1441757,Corrector activity at human bronchial epitheli...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,UNDEFINED MUTATION,[G551D],[G551D],P13569_G551D,G551D
...,...,...,...,...,...,...,...,...,...
374540,2127220,Binding affinity to 6His-FLAG-Tev-BRDT (1 to 3...,Q58F21,MSLPSRQTAIIVNPPPPEYINTKKNGRLTNQLQYLQKVVLKDLWKH...,UNDEFINED MUTATION,"[Y309A, Y66A]","[Y66A, Y309A]",Q58F21_Y66A_Y309A,Y66A;Y309A
374554,2127280,Inhibition of 6His-FLAG-Tev-BRDT (1 to 397 res...,Q58F21,MSLPSRQTAIIVNPPPPEYINTKKNGRLTNQLQYLQKVVLKDLWKH...,UNDEFINED MUTATION,"[Y309A, Y66A]","[Y66A, Y309A]",Q58F21_Y66A_Y309A,Y66A;Y309A
374556,2127282,Inhibition of 6His-Thr-BRD3 (1 to 435 residues...,Q15059,MSTATTVAPAGIPATPGPVNPPPPEVSNPSKPGRKTNQLQYMQNVV...,UNDEFINED MUTATION,"[Y348A, Y73A]","[Y73A, Y348A]",Q15059_Y73A_Y348A,Y73A;Y348A
374557,2127283,Inhibition of 6His-Thr-BRD2 (1 to 473 residues...,P25440,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,UNDEFINED MUTATION,"[Y386A, Y113A]","[Y113A, Y386A]",P25440_Y113A_Y386A,Y113A;Y386A


In [26]:
# Check number of assays per target if target is defined as 'accession' AND 'Protein_Type' (i.e. 'target_id')
chembl_assays_annotated.groupby(['accession','Protein_Type'])['assay_id'].count()

accession   Protein_Type
A0A024AXB9  WT              44
A0A045ISB3  F161S            1
            I203A            1
            I207A            1
            I207K            1
                            ..
W5VNF7      WT               5
W7JX98      T618Q            1
            WT               4
W8T2T2      WT               2
X5EH84      WT               5
Name: assay_id, Length: 9903, dtype: int64

In [27]:
# Check number of assays per target if target is defined as 'Protein_Type'
# i.e. all WT proteins together and every mutant separately
chembl_assays_annotated.groupby(['Protein_Type'])['assay_id'].count()

Protein_Type
A117V     1
A128T     2
A133D     6
A142G     3
A142T     3
         ..
Y93N      4
Y969C     7
Y97A     12
Y995A     1
Y99S      2
Name: assay_id, Length: 1748, dtype: int64

In [28]:
# Check number of assays per target if target is defined as 'accession'
chembl_assays_annotated.groupby(['accession'])['assay_id'].count()

accession
A0A024AXB9    44
A0A045ISB3    49
A0A059UE90     3
A0A060INS8    10
A0A072ZLE6    16
              ..
V9HWB9         5
W5VNF7         5
W7JX98         5
W8T2T2         2
X5EH84         5
Name: assay_id, Length: 8079, dtype: int64